# Building a Debugger with Cassette
<img src="https://avatars1.githubusercontent.com/u/46803805?s=156&v=4" style="display: inline"> <img src="https://raw.githubusercontent.com/jrevels/Cassette.jl/master/docs/img/cassette-logo.png" width="256" style="display: inline"/>
## Lyndon White (@oxinabox)
### RSE at Invenia Labs

In [1]:
function iprintstyled(xs...)
    color=last(xs)
    x = join(xs[1:end-1])
    display("text/html", """<pre style="color: $color">$x</pre>""")
end

iprintstyled (generic function with 1 method)

## With Thanks
 - Tim Holy
 - Kristoffer Carlsson
 - Jarrett Revels
 - Keno Fischer


## Outline:
 - About MagneticReadHead 
 - 3 Debuggers:
    - Overdubbing Calls only (MRH v0.1)
    - Track everything (MRH v0.2)
    - Track nothing (MRH v0.3)
    

### What is Magnetic Read Head ?

A Magentic Read Head sits above a cassette tape (or a magnetic disk), and reads the content off of it.

 - The MRH is a debugger that works by modifying the IR level code during the compilation process, to include debug instrumentation.
 - it is completely compiled using the standard julia compiler.
 - Incontrast to Debugger.jl which uses JuliaInterpreter.jl
 - Incontrast to Gallium which used some kind of LLVM magic

### Performance:
<span style="display: inline-block">

```julia
function summer(A)
   s = zero(eltype(A))
   for a in A
       s += a
   end
   return s
end
```
</span>    

<img src="benchmark.png" width="90%" style="display: inline-block"/>

## Performance of debuggers is complicated:
 - MRH has to recompile every method it touchs the first time,
    - run-only-once is common when debugging.
    - recompiling large libraries is huge slow
 - This kind of instrumentation destroys SIMD, and potentially breaks CPU pipelining

## Insane blackholes exist

```julia
julia> foo() = Complex.(rand(1,2), rand(1,2)) * rand(Int, 2,1);

julia> @btime foo();
  297.770 ns (9 allocations: 720 bytes)

julia> @btime Debugger.@run foo();
  15.472 ms (46982 allocations: 1.78 MiB)

julia> @time MagneticReadHead.@run foo()
  <Hangs for over 30 minutes>
```

## Julia IR: Lightning Intro

## Julia: layers of representenstation:
 - julia lowering:
     - Source code
     - AST: `quote`
     - Untyped IR: `@code_lowered`
     - Typed IR: `@code_typed`
     - LLVM: `@code_llvm`
     - ASM: `@code_native`

### Untyped IR: this is what we are working with
 - basically a linearization of the AST
 - Variables -> Slots
 - Loops -> Label + Goto
 - function names -> `GlobalRef(mod, func)`
 - Only 1 operation per statement (Nested expressions get broken up) 
 - the return values for each statement is accessed as `SSAValue(index)`

## How Does Cassette Work?
 - It is not magic, Cassette is not specially baked into the compiler.
 - `@generated` function can return a `Expr` **or** a `CodeInfo`
 - We return a `CodeInfo` based on a modified version of one for a function argument. Its a bit like a macro with dynamic scope.
 - This capacity allows on to build AD tools, Mocking tools, Debuggers and more.
     - Contrast: Swift for TensorFlow which is adding AD into the compiler.

### Manual pass
```julia
call_and_print(f, args...) = (println(f, " ", args); f(args...))

@generated function rewritten(f)
    meth = first(methods(f.instance, Tuple{}))
    ci = copy(Base.uncompressed_ast(meth))
    for ii in 1:length(ci.code)
        if ci.code[ii] isa Expr && ci.code[ii].head==:call
            func = GlobalRef(Main, :call_and_print)
            ci.code[ii] = Expr(:call, func, statement.args...)
        end
    end
    return ci
end
```

### Result of our manual pass:
```julia
julia> foo() = 2*(1+1);
julia> rewritten(foo)
+ (1, 1)
* (2, 2)
4
```

Rather than just transforming function calls to just call `call_and_print`  
we could have changed them to call `rewrite`,

This is how Cassette (and IRTools) work.

## Debugger 1
### MRH v0.1-like
### Overdub functions you want to debug.

### concept prototype

In [2]:
using Cassette
Cassette.@context Concept1
function Cassette.overdub(ctx::Concept1, f::typeof(+), args...)
    method = @which f(args...)
    iprintstyled("Breakpont Hit", :red)
    iprintstyled(method, :green);
    iprintstyled("Args: ", args, :blue);
    println("...press enter to continue...")
    #readline()
    
    Cassette.recurse(ctx, f, args...)
end

In [3]:
function foo(a)
    b = a+1
    c= 2b
    return b+c
end
Cassette.recurse(Concept1(), ()->foo(4))

Breakpont Hit

+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:53

Args: (4, 1)

Breakpont Hit

+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:53

Args: (5, 10)

...press enter to continue...
...press enter to continue...


15

### Generalizing that prototype
 - add setting of breakpoint via `@eval`
 - add deleting of breakpoints via `Base.deletemethod`
 - add Stepping via setting a breakpoint on every call
 - Improve UX via showing a REPL with named variables etc

In [4]:
Cassette.@context Concept2
function set_breakpoint(ctx::Concept2, f::F) where F
    @eval function Cassette.overdub(ctx::Concept2, f::$F, args...)
        method = @which f(args...)
        iprintstyled("Breakpont Hit", :red)
        iprintstyled(method, :green);
        iprintstyled("Args: ", args, :blue);
        println("...press enter to continue...")
        #readline()
        if Cassette.canrecurse(ctx, f, args...)
            Cassette.recurse(ctx, f, args...)
        else 
            Cassette.fallback(ctx, f, args...)
        end
    end
end

set_breakpoint (generic function with 1 method)

In [5]:
bar(x) = x + blarg(x)
blarg(x) = 5

set_breakpoint(Concept2(), blarg)

Cassette.@overdub Concept2() bar(1)

Breakpont Hit

blarg(x) in Main at In[5]:2

Args: (1,)

...press enter to continue...


6

## Debugger Bits
Lots of bits of a debugger are not too hard.

### A Decent REPL can be written in 50 lines
an ok one can be written in 12
```julia
function run_repl(name2var)
    while true
        code_ast = Meta.parse(readline())       # READ
        code_ast == nothing && break
        code_ast = subnames(name2var, code_ast)
        try
            result = eval(code_ast)             # EVAL
            display(result)                     # PRINT
        catch err
            showerror(stdout, err)
        end
    end                                         # LOOP
end
```


Fun Fact: Julia comes with 5 REPLs.

### CodeInfo `lineinfo` and `codelocs` contain all you need to male from Line to IR statement index

```julia
function src_line2ir_statement_ind(ir, src_line)
    linetable_ind = findlast(ir.linetable) do lineinfo
        lineinfo.line == src_line
    end
    statement_ind = findlast(isequal(linetable_ind), ir.codelocs)
    return statement_ind
end

function src_line2ir_statement_ind(meth::Method, src_line)
    ir = Base.uncompressed_ast(meth)
    return src_line2ir_statement_ind(ir, src_line)
end
```

### Use this to allow setting a breakpoint on a particular line.
CodeTracking.jl can help find where methods start and end

### Reverse: Finding Line Number from IR statement index  is even easier
```julia
function statement_ind2src_linenum(ir, statement_ind)
    code_loc = ir.codelocs[statement_ind]
    return ir.linetable[code_loc].line
end
```

### Use this to display breadcrumbs


### Working stepping state on exitting and entering function basically sucks
It is basically a state machine:
 - On the way is easy: 
     - 4 states: StepIn StepNext StepContinue StepOut
     
 - Out is harder:
     - 4*4 states, as the state before entering the called function and the state set by the called function need to be used.
     
 - this can all be controlled from the `overdub`

### Answer In (easy)
```julia
if Cassette.canrecurse(ctx, f, args...)
    if cur_mode === StepIn
        ctx.metadata.stepping_mode = StepNext
    else
        ctx.metadata.stepping_mode = StepContinue
    end
    try
        return Cassette.recurse(ctx, f, args...)
```

### Answer Out (Harder)
```julia
finally
    # Determine stepping mode for parent
    child_instruction = ctx.metadata.stepping_mode
    if child_instruction !== StepContinue
        ctx.metadata.stepping_mode = StepNext
    elseif cur_mode === StepIn
        ctx.metadata.stepping_mode = StepContinue
    else 
        ctx.metadata.stepping_mode = cur_mode
    end
end
```

## Debugger 2
### MRH v0.3-like
### Stop tracking things by hand

 - It is a bit redundant to be going aroud making copies of this that are already in memory **somewhere**
 - In the ideal world we would just insert a call to `@locals` into the lower code
    - but `@locals` is a fake macro -- it is actually resolved during lowering.
 - What we can do is insert the same kind of code that `@locals` would.

### To To this we are going to need to use an IR pass
 - IR pass is the same kind of thing we did at the start.
 - but Cassette gives us some helpers, and handles some of fiddlier things.
 - Your path function gets as its input a `Reflection` object.
 - It must return a `CodeInfo`

### What do we Know (`Reflection` input):
 - `method` (+ signature + static_params)
 - `code_info`: (copied) We are going to edit this
      - `code`: Array of linearized expressions (SSA form)
      - `codelocs`: maps from IR statement index to entry in `lineinfo`
      - `lineinfo`: tells you a position in linenumber + filenam
      - `slotnames`: Array of names for all the variables

In [38]:
Cassette.Reflection |> fieldnames

(:signature, :method, :static_params, :code_info)

In [41]:
ci = @code_lowered 1+1
dump(ci)

Core.CodeInfo
  code: Array{Any}((2,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: GlobalRef
          mod: Module Base
          name: Symbol add_int
        2: Core.SlotNumber
          id: Int64 2
        3: Core.SlotNumber
          id: Int64 3
    2: Expr
      head: Symbol return
      args: Array{Any}((1,))
        1: Core.SSAValue
          id: Int64 1
  codelocs: Array{Int32}((2,)) Int32[1, 1]
  method_for_inference_limit_heuristics: Nothing nothing
  ssavaluetypes: Int64 2
  linetable: Array{Any}((1,))
    1: Core.LineInfoNode
      mod: Module Base
      method: Symbol +
      file: Symbol int.jl
      line: Int64 53
      inlined_at: Int64 0
  ssaflags: Array{UInt8}((0,)) UInt8[]
  slotflags: Array{UInt8}((3,)) UInt8[0x00, 0x00, 0x00]
  slotnames: Array{Any}((3,))
    1: Symbol #self#
    2: Symbol x
    3: Symbol y
  inferred: Bool false
  inlineable: Bool false
  propagate_inbounds: Bool false
  pure: Bool false


In [7]:
Cassette.@context Concept31
function Cassette.overdub(ctx::Concept31, f, args...)
    depth = ctx.metadata
    if depth[] < 3 && Cassette.canrecurse(ctx, f, args...)
        method = @which f(args...)
        iprintstyled(method, :green);
        depth[] += 1
        try
            Cassette.recurse(ctx, f, args...)
        finally
            depth[] -= 1
        end
    else 
        Cassette.fallback(ctx, f, args...)
    end
end


In [12]:
call_expr(mod::Module, func::Symbol, args...) = Expr(:call, Expr(:nooverdub, GlobalRef(mod, func)), args...)

function instrument31!(::Type{<:Concept31}, reflection::Cassette.Reflection)
    ci = reflection.code_info
    
    enter_debug_statements(::Any, ::Any) = nothing
    function enter_debug_statements(stmt::Expr, initial_ind)
        cur_ind=initial_ind
        new_statements=[]
        for (slotind, slotname) in enumerate(ci.slotnames)
            slot = Core.SlotNumber(slotind)
            push!(new_statements, Expr(:isdefined, slot))
            cur_ind+=1
            push!(new_statements,
                Expr(:gotoifnot, Core.SSAValue(cur_ind-1), cur_ind + 2)
            )
            cur_ind+=1
            push!(new_statements, 
                call_expr(Main, :iprintstyled,
                    "Index ", initial_ind, " ",
                    QuoteNode(slotname), " ", slot,
                    QuoteNode(:blue)
                )
            )
            cur_ind+=1
        end
        push!(new_statements, stmt)
        return new_statements
    end
    
    Core.println()
    Cassette.insert_statements!(
        ci.code, ci.codelocs,
        (stmt, i) -> stmt isa Expr ? 3length(ci.slotnames)+1 : nothing,
        enter_debug_statements,
    )
    #Core.println(ci)
    Core.println
    return ci
end

pass31n = Cassette.@pass instrument31!

getfield(Main, Symbol("##PassType#364"))()

In [36]:
function fun_times()
    y=2
    x=2
    y=x+y
end 
Cassette.@overdub Concept31(metadata=Ref(0), pass=pass31n) fun_times()

Index 1 #self# getfield(Main, Symbol("##68#69"))()

fun_times() in Main at In[36]:2

Index 1 #self# fun_times

Index 11 #self# fun_times

Index 11 y 2

...

Index 41 #self# fun_times

Index 41 x 2

Index 41 y 4

Index 5 #self# getfield(Main, Symbol("##68#69"))()

4

In [35]:
function danger19()
    y=2
    function inner()
        h=y
        y=12
        return h
    end
    inner()
end 
Cassette.@overdub Concept31(metadata=Ref(0), pass=pass31n) danger19()

Index 1 #self# getfield(Main, Symbol("##66#67"))()

danger19() in Main at In[35]:2

Index 1 #self# danger19

Core.Box() in Core at boot.jl:337

Index 1 #self# Core.Box

...

Index 41 y Core.Box(12)

Index 41 inner getfield(Main, Symbol("#inner#65"))(Core.Box(12))

Index 5 #self# getfield(Main, Symbol("##66#67"))()

Internal error: encountered unexpected error in runtime:
BoundsError(a=Array{Any, (80,)}[SSAValue(1), SSAValue(2), SSAValue(3), SSAValue(18446744073709551615), SSAValue(5), SSAValue(6), SSAValue(18446744073709551615), SSAValue(8), SSAValue(18446744073709551615), SSAValue(10), SSAValue(11), SSAValue(12), SSAValue(13), SSAValue(14), SSAValue(18446744073709551615), SSAValue(16), SSAValue(17), SSAValue(18446744073709551615), SSAValue(19), SSAValue(18446744073709551615), SSAValue(21), SSAValue(22), SSAValue(23), SSAValue(24), SSAValue(25), SSAValue(18446744073709551615), SSAValue(27), SSAValue(28), SSAValue(18446744073709551615), SSAValue(30), SSAValue(18446744073709551615), SSAValue(18446744073709551615), SSAValue(18446744073709551615), SSAValue(34), SSAValue(35), SSAValue(36), SSAValue(18446744073709551615), SSAValue(38), SSAValue(39), SSAValue(18446744073709551615), SSAValue(41), SSAValue(18446744073709551615), SSAValue(43), SSAValue(44), SSAValue(45), SSAValue(46), SSAValue(47), SSAValu

2

In [31]:
function danger11(x)
    if x==1
        y=1
        return x
    elseif x==2
        let
            m=2
            x+=m
        end
        return 2x
    end
    return x
end

Cassette.@overdub Concept31(metadata=Ref(0), pass=pass31n) danger11(1)

Index 1 #self# getfield(Main, Symbol("##54#55"))()

danger11(x) in Main at In[31]:2

Index 1 #self# danger11

Index 1 x 1

Index 18 #self# danger11

...

Index 66 y 1

Index 66 x 1

Index 5 #self# getfield(Main, Symbol("##54#55"))()

Internal error: encountered unexpected error in runtime:
BoundsError(a=Array{Any, (187,)}[SSAValue(1), SSAValue(2), SSAValue(3), SSAValue(18446744073709551615), SSAValue(5), SSAValue(6), SSAValue(18446744073709551615), SSAValue(8), SSAValue(9), SSAValue(18446744073709551615), SSAValue(11), SSAValue(18446744073709551615), SSAValue(13), SSAValue(18446744073709551615), SSAValue(15), SSAValue(16), SSAValue(17), SSAValue(18446744073709551615), SSAValue(19), SSAValue(20), SSAValue(18446744073709551615), SSAValue(22), SSAValue(23), SSAValue(18446744073709551615), SSAValue(25), SSAValue(18446744073709551615), SSAValue(27), SSAValue(18446744073709551615), SSAValue(29), SSAValue(30), SSAValue(31), SSAValue(32), SSAValue(33), SSAValue(34), SSAValue(18446744073709551615), SSAValue(36), SSAValue(37), SSAValue(18446744073709551615), SSAValue(39), SSAValue(40), SSAValue(18446744073709551615), SSAValue(42), SSAValue(18446744073709551615), SSAValue(44), SSAValue(18446744073709551615), SSAValue(46), SSA

1

In [11]:
?Base.isassigned

```
isassigned(array, i) -> Bool
```

Test whether the given array has a value associated with index `i`. Return `false` if the index is out of bounds, or has an undefined reference.

# Examples

```jldoctest
julia> isassigned(rand(3, 3), 5)
true

julia> isassigned(rand(3, 3), 3 * 3 + 1)
false

julia> mutable struct Foo end

julia> v = similar(rand(3), Foo)
3-element Array{Foo,1}:
 #undef
 #undef
 #undef

julia> isassigned(v, 1)
false
```


# What is next for MagneticReadHead ? 
 - Bug squashing
 - Rewrite in IRTools.jl
 - Think about meta-debugging instrumentation.
 - More alignment with Debugger.jl
 - /Apply this kind of tooling into Debugger.jl

 